In [ ]:
%pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error

import xgboost as xgb

color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [ ]:
df = pd.read_csv("hourly-energy-consumption/PJME_hourly.csv")
df = df.set_index("Datetime")
df.index = pd.to_datetime(df.index)
df = df.sort_index()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.plot(style=".",
        figsize=(15,5),
        color=color_pal[0],
        title="PJME Energy Use in MW")

In [ ]:
train = df.loc[df.index < "01-01-2015"]
test = df.loc[df.index >= "01-01-2015"]

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))

train.plot(ax=ax, label="Training Set", title="Data Train/Test Split")
test.plot(ax=ax, label="Test Set")

ax.axvline("01-01-2015", color="black", ls="--")
ax.legend(["Training Set, Test Set"])

In [ ]:
df.loc[(df.index > "01-01-2010") & (df.index < "01-08-2010")].plot(figsize=(15,5), title="Week of Data")

In [ ]:
def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()

    df["hour"] = df.index.hour
    df["dayofweek"] = df.index.dayofweek
    df["quarter"] = df.index.quarter
    df["month"] = df.index.month
    df["year"] = df.index.year
    df["dayofyear"] = df.index.dayofyear

    return df

In [ ]:
df = create_features(df=df)

In [ ]:
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

sns.boxplot(data=df, x="hour", y="PJME_MW")

ax.set_title("MW by Hour")

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

sns.boxplot(data=df, x="month", y="PJME_MW")

ax.set_title("MW by Month")

In [ ]:
train = create_features(train)
test = create_features(test)

FEATURES = ['hour', 'dayofweek', 'quarter', 'month', 'year', 'dayofyear']
TARGET = ['PJME_MW']

In [ ]:
X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

In [ ]:
reg = xgb.XGBRegressor(n_estimators=5000,
                       early_stopping_rounds=50,
                       learning_rate=0.001,
                       n_jobs=-1)

reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train),(X_test, y_test)],
        verbose=300)

In [ ]:
fi = pd.DataFrame(data=reg.feature_importances_,
             index=reg.feature_names_in_,
             columns=["importance"])

In [ ]:
fi.sort_values("importance").plot(kind="barh", title="Feature Importance")

In [ ]:
test["prediction"] = reg.predict(X_test)
df = df.merge(test[["prediction"]], how="left", left_index=True, right_index=True)

In [ ]:
ax = df[["PJME_MW"]].plot(figsize=(15,5))
df["prediction"].plot(ax=ax, style=".")

plt.legend(["Truth Data", "Predictions"])
ax.set_title("Raw Data and Predictions")

In [ ]:
ax = df.loc[(df.index > "01-01-2018") & (df.index < "01-08-2018")]["PJME_MW"] \
    .plot(figsize=(15,5), title="Week of Data")
df.loc[(df.index > "01-01-2018") & (df.index < "01-08-2018")]["prediction"] \
    .plot(style=".")

plt.legend(["Truth Data", "Predictions"])

In [ ]:
score = np.sqrt(mean_squared_error(test["PJME_MW"], test["prediction"]))
print(f"RMSE Score on Test Set: {score:0.2f}")

In [ ]:
test["error"] = np.abs(test["PJME_MW"] - test["prediction"])

In [ ]:
test["date"] = test.index.date

In [ ]:
test.groupby("date")["error"].mean().sort_values(ascending=False).head(10)